In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd
import os
import scipy.stats as stats

# Funcion para calcular la energia promedio de un canal
Recibe como argumentos la ruta/nombre del archivo que contiene la señal y retorna la energia promedio del canal

In [9]:
def calcular_energia_promedio_por_canal(archivo_mat):
    # Cargar el archivo .mat
    data = sio.loadmat(archivo_mat)

    # Extraer la señal EEG
    eeg_data = data['data']

    # Calcular la energía por cada canal (suma de los cuadrados de los valores)
    energia_por_canal = np.mean(np.sum(eeg_data ** 2, axis=2), axis=0)
    
    return energia_por_canal


archivo_control = 'control/C001R_EP_reposo.mat'
archivo_parkinson = 'parkinson/P001_EP_reposo.mat'
energia_control = calcular_energia_promedio_por_canal(archivo_control)
energia_parkinson = calcular_energia_promedio_por_canal(archivo_parkinson)

print("Energía promedio por canal (Control):", energia_control)
print("Energía promedio por canal (Parkinson):", energia_parkinson)


Energía promedio por canal (Control): [2529.73030796 2385.72046929 2251.40359349 ... 2003.96200576 2089.37463374
 2183.16700378]
Energía promedio por canal (Parkinson): [1270.68018168 1214.46053902 1164.59365832 ...  972.48312595 1015.75237627
 1063.35518041]


# Energia promedio de cada canal para ambos grupos (control y parkinson) y creacion de cada dataframe (por grupo) con la lista de las energias promedios de cada sujeto

In [10]:
def obtener_dataframe_energia(directorio, grupo):
    archivos = [f for f in os.listdir(directorio) if f.endswith('.mat')]
    energias = []

    for archivo in archivos:
        ruta_archivo = os.path.join(directorio, archivo)
        
        # Calcular la energía promedio por canal
        energia_canal = calcular_energia_promedio_por_canal(ruta_archivo)
        energias.append(energia_canal)
    
    # Crear un DataFrame con los resultados
    df_energia = pd.DataFrame(energias, columns=[f'Canal_{i+1}' for i in range(energia_canal.shape[0])])
    
    # Añadir una columna con el nombre de los sujetos
    df_energia.index = [f'Sujeto_{i+1}' for i in range(len(archivos))]
    
    return df_energia

# Calcular las energías para los grupos de control y parkinson
directorio_control = 'control'
directorio_parkinson = 'parkinson'

df_energia_control = obtener_dataframe_energia(directorio_control, 'control')
df_energia_parkinson = obtener_dataframe_energia(directorio_parkinson, 'parkinson')

# Guardar los resultados en archivos CSV
df_energia_control.to_csv('energia_control.csv')
df_energia_parkinson.to_csv('energia_parkinson.csv')

print("Energía promedio por canal - Grupo Control")
print(df_energia_control)

print("\nEnergía promedio por canal - Grupo Parkinson")
print(df_energia_parkinson)

Energía promedio por canal - Grupo Control
                Canal_1       Canal_2       Canal_3       Canal_4  \
Sujeto_1    2529.730308   2385.720469   2251.403593   2130.312696   
Sujeto_2    3902.726731   3829.824868   3762.011356   3702.140295   
Sujeto_3    1540.020453   1466.807011   1401.486001   1345.115149   
Sujeto_4    5921.007972   5823.011917   5728.652940   5640.572302   
Sujeto_5    2666.604596   2581.562375   2497.638854   2417.729488   
Sujeto_6    1885.315270   1813.995948   1748.003414   1689.232177   
Sujeto_7    1115.164069   1060.089071   1009.539288    964.281734   
Sujeto_8    3037.189210   2971.538992   2910.079182   2854.890564   
Sujeto_9    2116.521620   2076.889937   2038.665256   2002.253279   
Sujeto_10  11838.843945  11732.199031  11629.432948  11536.379469   
Sujeto_11   3682.341763   3505.468704   3336.431488   3180.784471   
Sujeto_12  10035.066086   9923.424717   9807.649395   9691.877363   
Sujeto_13   8139.175918   8120.123278   8109.812767   8112.9

# Comparacion de los canales de cada grupo

In [11]:
def comparar_canales(df_control, df_parkinson):
    resultados = []

    for canal in df_control.columns:
        # Energías de cada canal en los dos grupos
        energia_control = df_control[canal]
        energia_parkinson = df_parkinson[canal]

        # Verificar normalidad con prueba de Shapiro-Wilk
        stat_control, p_control = stats.shapiro(energia_control)
        stat_parkinson, p_parkinson = stats.shapiro(energia_parkinson)
        normalidad_control = p_control > 0.05
        normalidad_parkinson = p_parkinson > 0.05

        # Verificar homocedasticidad con prueba de Levene
        stat_levene, p_levene = stats.levene(energia_control, energia_parkinson)
        homocedasticidad = p_levene > 0.05

        # Si ambos grupos cumplen con la prueba de normalidad y tienen varianza similar, aplicamos la prueba t
        if normalidad_control and normalidad_parkinson and homocedasticidad:
            stat_t, p_t = stats.ttest_ind(energia_control, energia_parkinson, equal_var=True)
            metodo = 'Prueba t'
        else:
            # Si no se cumple normalidad o varianza igual, usamos Mann-Whitney
            stat_t, p_t = stats.mannwhitneyu(energia_control, energia_parkinson)
            metodo = 'Prueba U de Mann-Whitney'

        # Guardar los resultados para este canal
        resultados.append({
            'Canal': canal,
            'Normalidad Control': normalidad_control,
            'Normalidad Parkinson': normalidad_parkinson,
            'Homocedasticidad': homocedasticidad,
            'Método': metodo,
            'p-valor': p_t
        })

    # Convertir los resultados a DataFrame
    df_resultados = pd.DataFrame(resultados)
    return df_resultados

# Ejecutar la función
df_resultados_comparacion = comparar_canales(df_energia_control, df_energia_parkinson)
print(df_resultados_comparacion)
# Guardar los resultados en un CSV
df_resultados_comparacion.to_csv('resultados_comparacion_canales.csv', index=False)

           Canal  Normalidad Control  Normalidad Parkinson  Homocedasticidad  \
0        Canal_1               False                 False              True   
1        Canal_2               False                 False              True   
2        Canal_3               False                 False              True   
3        Canal_4               False                 False              True   
4        Canal_5               False                 False              True   
...          ...                 ...                   ...               ...   
1995  Canal_1996               False                 False              True   
1996  Canal_1997               False                 False              True   
1997  Canal_1998               False                 False              True   
1998  Canal_1999               False                 False              True   
1999  Canal_2000               False                 False              True   

                        Método   p-valo

# Conclusiones

- Normalidad de los datos: La prueba de normalidad para ambos grupos (Control y Parkinson) indica que ninguno de los canales sigue una distribución normal, ya que todos los canales tienen un resultado de "False" en ambas columnas ("Normalidad Control" y "Normalidad Parkinson"). Esto significa que las señales de EEG de ambos grupos no se distribuyen de forma normal.

- Homocedasticidad (igualdad de varianzas): Es cumplida para todos los canales, ya que todos los valores en la columna "Homocedasticidad" son "True". Aunque los datos no son normales, este resultado sugiere que las varianzas de las señales de EEG son similares entre los grupos de control y Parkinson para todos los canales.

Nota: Dado que no se cumple la normalidad en los datos, se utiliza la prueba U de Mann-Whitney para comparar los canales entre los grupos.

- Resultados de la prueba U de Mann-Whitney: Los p-valores de la prueba U de Mann-Whitney para todos los canales están en rangos superiores a 0.05 (ej: 0.414533, 0.405701, etc.), lo que indica que no se encuentran diferencias estadísticamente significativas entre los grupos de control y Parkinson para ninguno de los canales. Como todos los p-valores son mayores al umbral típico (0.05), no podemos rechazar la hipótesis nula. Esto implica que no hay evidencia suficiente para afirmar que existe una diferencia en la energía de los canales de EEG entre los pacientes con Parkinson y el grupo control.

No hay diferencias significativas en la energía de los canales de EEG entre los pacientes con Parkinson y los sujetos control. Este resultado sugiere que la energía de las señales no proporciona una distinción clara entre estos dos grupos cuando se analiza de manera agregada por canal. La falta de normalidad en los datos de ambos grupos podría ser una característica relevante de las señales de EEG, lo que podría implicar la necesidad de utilizar técnicas no paramétricas o transformaciones adicionales para detectar posibles diferencias en otras características. Por otro lado, el hecho de que se cumpla la homocedasticidad indica que, aunque los grupos tienen distribuciones distintas, las fluctuaciones en la energía de los canales son similares entre ambos grupos. Por ultimo, la ausencia de diferencias significativas en la energía de los canales podría sugerir que la energía por sí sola no es una característica que discrimine entre los dos grupos, por lo que sería necesario explorar otras características o combinaciones de métricas para identificar posibles patrones diferenciales entre pacientes con Parkinson y sujetos sanos.

En base a los resultados obtenidos, no se identificaron canales de EEG que proporcionen información diferencial significativa en términos de energía entre los grupos de control y Parkinson. Esto sugiere que la energía de los canales, por sí sola, no es una característica lo suficientemente fuerte para discriminar entre estos dos grupos, al menos para la base de datos que se esta usando.

# Referencias
- https://datatab.es/tutorial/test-of-normality
- https://datatab.es/tutorial/levene-test
- https://datatab.es/tutorial/t-test
- https://datatab.es/tutorial/mann-whitney-u-test